# Pandas 学习笔记

In [1]:
import numpy as np
import pandas as pd

## 数据结构简介

### Series的定义及用法
`Series`是一维标记的数组，能够保存任何数据类型。轴标签统称为**索引**。创建`Series`的基本方法是调用以下函数：

`s = pd.Series(data, index=index)`

如果没有传入索引，则将自动创建`range(len(data))`的数值序列作为索引。  

`Series`可直接从`dicts`对象创建数据列表，例如：

In [ ]:
d = {'b': 1, 'a': 0, 'c': 2}

In [ ]:
# Python3.7版本在创建数据表时按照dict传入顺序进行排序
pd.Series(d)

In [ ]:
# 如果传递索引，则将索引中的标签与dict中的key值直接对应
pd.Series(d, index=['b', 'c', 'd', 'a'])

In [ ]:
# 如果data是标量值，则必须提供索引，且创建时重复该值以匹配索引长度
pd.Series(5., index=['a', 'b', 'c', 'd', 'e'])

### Series切片

In [ ]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

使用类似`list`的切片方法：

In [ ]:
s[0]

In [ ]:
s[:3]

使用基于数组的切片：

In [ ]:
s[[4,3,1]]

条件筛选：

In [ ]:
s[s>0]

使用类似`dict`的访问方法：

In [ ]:
s['a']

In [ ]:
s.a

In [ ]:
'e' in s

In [ ]:
# 可使用get方法访问某标签对应的值，不存在时则无任何返回值
s.get('e')

In [ ]:
# f标签不存在，设置不存在时返回的值为NaN
s.get('f', np.nan)

`Series`和`ndarray`之间的主要区别在于`Series`之间的操作会根据标签自动对齐数据。因此可以在不考虑所涉及的`Series`是否具有相同标签的情况下编写计算。未对齐`Series`之间的操作结果将包含所涉及的索引的并集。如果在一个`Series`或另一个`Series`中找不到标签，则结果将标记为缺失`NaN`，例如：

In [ ]:
s[1:] + s[:-1]

其他操作：

In [ ]:
# 查看数据类型
s.dtype

In [ ]:
# 转换为 ndarray
s.to_numpy()

`Series`的名称属性:

In [ ]:
s = pd.Series(np.random.randn(5), name='something')

s.name

In [ ]:
# 可使用rename方法重命名，此方法会复制原数据表，并赋给一个新的变量
s2 = s.rename("different")
s2.name

### DataFrame数据帧

`DataFrame`是一个二维标记数据结构，具有可能不同类型的列，类似于SQL表。  

除了数据，还可以选择传递**索引**（行标签）和**列参数**（列标签）。如果未传递轴标签，则将自动根据常识规则从输入数据构造它们。  

从`dict`构造数据帧：

In [2]:
d = {'one': pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
     'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])}

df = pd.DataFrame(d)

df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


In [ ]:
# 访问DataFrame的索引
df.index

In [ ]:
# 访问列参数
df.columns

从结构化数组创建数据帧：

In [ ]:
data = np.zeros((2, ), dtype=[('A', 'i4'), ('B', 'f4'), ('C', 'a10')])
data[:] = [(1, 2., 'Hello'), (2, 3., "World")]

pd.DataFrame(data)

### 替代构造函数

`DataFrame.from_dict`采用dicts的dict或类似数组序列的dict并返回DataFrame：

In [ ]:
pd.DataFrame.from_dict(dict([('A', [1, 2, 3]), ('B', [4, 5, 6])]))

In [3]:
# 加入 orient='index'，可以继续传递列名
pd.DataFrame.from_dict(dict([('A', [1, 2, 3]), ('B', [4, 5, 6])]),
                       orient='index', columns=['one', 'two', 'three'])

,one,two,three
A,1,2,3
B,4,5,6


`DataFrame.from_records`获取元组列表或带有结构化dtype的ndarray。它类似于普通DataFrame构造函数，但生成的DataFrame索引可能是结构化dtype的特定字段。

In [ ]:
pd.DataFrame.from_records(data, index='C')

列选择、添加、删除：

In [ ]:
df['one']

In [4]:
# 直接创建新列
df['three'] = df['one'] + df['two']

In [5]:
# 将条件筛选后的Series赋值给新列
df['flag'] = df['one'] > 2

In [ ]:
# 删除某列
del df['flag']

In [ ]:
# 删除某列并将删除内容返回赋给新的变量
three = df.pop('three')

In [ ]:
# 在指定位置插入数据
df.insert(1, 'bar', df['one'])

使用`assign`方法生成新列，但该方法始终返回数据的副本，而不改变原数据。

In [ ]:
# 创建从现有列派生的新列
df.assign(ratio1=df['one']/df['two'])

In [ ]:
# 给assign传入函数进行计算
df.assign(ratio2=lambda x: (x['one']/x['two']))

In [ ]:
# 创建新列时可同时创建不同的列，并且使用前一步所得到的内容
df.assign(four=lambda x: x['one'] + x['two'],
          five=lambda x: x['one'] + x['four'])

该方法常用于在不改变原数据的情况下进行相关计算和可视化，例如：
```
>>>iris.query('SepalLength > 5')
       .assign(SepalRatio=lambda x: x.SepalWidth / x.SepalLength,
               PetalRatio=lambda x: x.PetalWidth / x.PetalLength)
       .plot(kind='scatter', x='SepalRatio', y='PetalRatio')
```

## 索引/选择

选择某行/列/切片：

In [ ]:
# 列选择 - 通过列标签
df['one']

In [ ]:
# 行选择 - 通过索引（行标签）
df.loc['a']

In [ ]:
# 行选择 - 通过行号（整数位置）
df.iloc[1]

In [ ]:
# 行切片
df[1:3]

选择表中特定位置的值：

In [ ]:
# 获取特定位置的值 - 通过索引和列标签
df.at['a','one']

In [ ]:
# 获取特定位置的值 - 通过整数坐标
df.iat[3,1]

布尔数选择：

In [ ]:
df[df>0]

In [ ]:
df[(df>=1) | (df<2)]

In [ ]:
df[~(df>0)]

In [ ]:
# 此处仅支持列筛选
df[df['one']>0]

In [ ]:
# 筛选后将不满足的位置重新赋值
df2 = df.copy()
df2[df2['one']>0] = "YES"
df2

范围选择：

In [ ]:
df.isin([1,2])

`where()` 函数：

In [ ]:
# where函数始终返回数据的复制
df.where(df>0)

In [ ]:
# 第二个参数表明，当某值不符合条件时所返回的内容，此内容中为值的相反数
df.where(df<0, -df)

In [ ]:
# 修改原数据而非返回其复制
df.where(df<0, -df, inplace=True)

`mask()` 函数，常用于遮盖符合特定条件的值：

In [ ]:
df.mask(df>=0)

`query()` 函数

In [ ]:
df.query('one<=two | two>bar')

## 数据采样

In [ ]:
# 随机选取一行数据
df.sample()

In [ ]:
# 指定采样数量
df.sample(n=3)

In [ ]:
# 按比例抽样
df.sample(frac=0.5)

In [ ]:
# 加权抽样
example_weights = [0.1, 0.2, 0.3, 0.4]
df.sample(n=2, weights=example_weights)

## 基本使用方法

下列使用的数据结构如下：

In [ ]:
index = pd.date_range('20190101', periods=8)
s = pd.Series(np.random.randn(5), index=['AA', 'BB', 'CC', 'DD', 'EE'])

df = pd.DataFrame(np.random.randn(8,3), index=index, columns=['A', 'B', 'C'])

#查看df内容
df

In [ ]:
# 预览前5条数据，可指定显示数量
df.head()

In [ ]:
# 预览最后5条数据
df.tail(3)

In [18]:
# 查看数据类型概况
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 4 columns):
one      3 non-null float64
two      4 non-null float64
three    3 non-null float64
flag     4 non-null bool
dtypes: bool(1), float64(3)
memory usage: 132.0+ bytes


In [19]:
# 查看数据统计概况
df.describe()

,one,two,three
count,3.0,4.000000,3.0
mean,2.0,2.500000,4.0
std,1.0,1.290994,2.0
min,1.0,1.000000,2.0
25%,1.5,1.750000,3.0
50%,2.0,2.500000,4.0
75%,2.5,3.250000,5.0
max,3.0,4.000000,6.0


In [15]:
df.shape

(4, 4)

In [9]:
df.to_numpy()

array([[1.0, 1.0, 2.0, False],
       [2.0, 2.0, 4.0, False],
       [3.0, 3.0, 6.0, True],
       [nan, 4.0, nan, False]], dtype=object)

In [11]:
df.to_string()

'   one  two  three   flag\na  1.0  1.0    2.0  False\nb  2.0  2.0    4.0  False\nc  3.0  3.0    6.0   True\nd  NaN  4.0    NaN  False'

## Pandas 其他内置函数

In [13]:
# 设置显示宽度
pd.set_option('display.width', 40)

In [ ]:
# 设置各列的最大宽度
pd.set_option('display.max_clowidth', 30)